In [ ]:
#default_exp geometry.geometry

In [ ]:
# export
def get_geom(data, radius, n_neighbors):
    bigradius = 3 * radius
    adjacency_method = 'cyflann'
    cyflann_kwds = {'index_type':'kdtrees', 'num_trees':10, 'num_checks':n_neighbors}
    adjacency_kwds = {'radius':bigradius, 'cyflann_kwds':cyflann_kwds}
    affinity_method = 'gaussian'
    affinity_kwds = {'radius':radius}
    laplacian_method = 'geometric'
    laplacian_kwds = {'scaling_epps':radius}
    geom = Geometry(adjacency_method=adjacency_method, adjacency_kwds=adjacency_kwds,
                    affinity_method=affinity_method, affinity_kwds=affinity_kwds,
                    laplacian_method=laplacian_method, laplacian_kwds=laplacian_kwds)
    geom.set_data_matrix(data)
    adjacency_matrix = geom.compute_adjacency_matrix()
    laplacian_matrix = geom.compute_laplacian_matrix()  
    return(geom)



In [2]:
# export 
from megaman.geometry import RiemannMetric
from einops import rearrange
import numpy as np

def get_rm_tangent_sel(embedding, geom, selectedpoints, d):
    
    #can we do this on subset of points?
    rmetric = RiemannMetric(embedding,geom.laplacian_matrix,n_dim=d)
    rmetric.get_rmetric()    
    tangent_bases = rearrange(rmetric.Hvv,'i b1 b2 -> i b2 b1')
    #tangent_bases = np.swapaxes(rmetric.Hvv[:,:d,:],1,2)
    return(tangent_bases)
    
def get_nbr_wts(A, sample):
    Ps = list()
    nbrs = list()
    for ii in range(len(sample)):
        w = np.array(A[sample[ii],:].todense()).flatten()
        p = w / np.sum(w)
        nbrs.append(np.where(p > 0)[0])
        Ps.append(p[nbrs[ii]])
    return(Ps, nbrs)

def get_wlpca_tangent_sel(data, geom, selectedpoints, d):

    #n = self.n
    b = data.shape[1]
    
    nsel = len(selectedpoints)
    A = geom.affinity_matrix
    (PS, nbrs) = get_nbr_wts(A, selectedpoints)
    
    tangent_bases = np.zeros((nsel, b, d))
    for i in range(nsel):
        # print(i)
        p = PS[i]
        nbr = nbrs[i]
        Z = (data[nbr, :] - np.dot(p, data[nbr, :])) * p[:, np.newaxis]
        sig = np.dot(Z.transpose(), Z)
        e_vals, e_vecs = np.linalg.eigh(sig)
        j = e_vals.argsort()[::-1]  # Returns indices that will sort array from greatest to least.
        e_vec = e_vecs[:, j]
        e_vec = e_vec[:, :d]
        tangent_bases[i, :, :] = e_vec
    return (tangent_bases)


In [ ]:
# export
import matplotlib.pyplot as plt
from megaman.embedding import spectral_embedding
from megaman.geometry import Geometry
from megaman.geometry import RiemannMetric
from scipy import sparse
from scipy.sparse.linalg import norm
from mpl_toolkits.mplot3d import proj3d
#from codes.geometer import TangentBundle
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

class RiemannianManifold:

    def __init__(self, data, dim):
        self.data = data
        self.n = data.shape[0]
        self.dim = dim

    def compute_geom(self, radius, n_neighbors):
        data = self.data
        dim = self.dim
        #set radius according to paper (i think this is dim not d)
        #radius = (diffusion_time * (diffusion_time * np.pi * 4)**(dim/2))**(0.5)
        #set adjacency radius large enough that points beyond it have affinity close to zero
        bigradius = 3 * radius
        adjacency_method = 'cyflann'
        cyflann_kwds = {'index_type':'kdtrees', 'num_trees':10, 'num_checks':n_neighbors}
        adjacency_kwds = {'radius':bigradius, 'cyflann_kwds':cyflann_kwds}
        affinity_method = 'gaussian'
        affinity_kwds = {'radius':radius}
        laplacian_method = 'geometric'
        laplacian_kwds = {'scaling_epps':radius}
        geom = Geometry(adjacency_method=adjacency_method, adjacency_kwds=adjacency_kwds,
                        affinity_method=affinity_method, affinity_kwds=affinity_kwds,
                        laplacian_method=laplacian_method, laplacian_kwds=laplacian_kwds)
        geom.set_data_matrix(data)
        adjacency_matrix = geom.compute_adjacency_matrix()
        laplacian_matrix = geom.compute_laplacian_matrix()
        return(geom)

    def compute_geom_brute(self, diffusion_time, n_neighbors):
        data = self.data
        dim = self.dim
        #set radius according to paper (i think this is dim not d)
        radius = (diffusion_time * (diffusion_time * np.pi * 4)**(dim/2))**(0.5)
        #set adjacency radius large enough that points beyond it have affinity close to zero
        bigradius = 3 * radius
        adjacency_method = 'brute'
        adjacency_kwds = {'radius':bigradius}
        affinity_method = 'gaussian'
        affinity_kwds = {'radius':radius}
        laplacian_method = 'geometric'
        laplacian_kwds = {'scaling_epps':1}
        geom = Geometry(adjacency_method=adjacency_method, adjacency_kwds=adjacency_kwds,
                        affinity_method=affinity_method, affinity_kwds=affinity_kwds,
                        laplacian_method=laplacian_method, laplacian_kwds=laplacian_kwds)
        geom.set_data_matrix(data)
        geom.adjacency_matrix = geom.compute_adjacency_matrix()
        geom.laplacian_matrix = geom.compute_affinity_matrix()
        geom.laplacian_matrix = self.get_laplacian(geom, radius)
        return(geom)

    def get_laplacian(self, geom, rad):
        n = geom.affinity_matrix.shape[0]
        x = np.squeeze(geom.affinity_matrix.sum(axis = 1))
        y = sparse.spdiags(x, 0, x.size, x.size)
        yinv = sparse.linalg.inv(y)
        tildepp = yinv @ geom.affinity_matrix @ yinv
        tildex = np.squeeze(tildepp.sum(axis = 1))
        tildey = sparse.spdiags(tildex, 0, tildex.size, tildex.size)
        tildeyinv = sparse.linalg.inv(tildey)
        lapland = (sparse.identity(n) - tildeyinv @ tildepp)
        lb = 4* lapland / (rad**2)
        return(lb)

    def compute_spectral_embedding(self, geom):
        geom.Y0, eigenvalues, eigenvectors = spectral_embedding.spectral_embedding(geom=geom, eigen_solver='amg', random_state=6, diffusion_maps=True, diffusion_time = diffusion_time, n_components=n_components)

    def get_induced_rmetric(self, embedding, geom, selected_points):
        dim = self.dim
        laplacian = geom.laplacian_matrix
        laplacian = laplacian.tocsr()[selected_points].tocoo()
        geom.rmetric = RiemannMetric(embedding[selected_points,:],laplacian,n_dim=dim)
        geom.rmetric.get_rmetric()
        return(geom)

#     def plot(self, axes, selected_points, c,s,alpha,filename, cbar = True):

#         data = self.data
#         if len(axes) == 2:
#             fig = plt.figure(figsize=plt.figaspect(.5))
#             ax = fig.add_subplot(1, 1,1)
#             cax = ax.scatter(data[:, axes[0]][selected_points], data[:, axes[1]][selected_points],  c = c, s=  s, alpha=alpha, marker = '.')
#             ax.set_axis_off()
#             lims  =np.max(data, axis = 0)
#             q1 = ax.quiver(lims[0], 0, angles='xy', scale_units='xy', scale=1)
#             q2 = ax.quiver( 0,lims[1], angles='xy', scale_units='xy', scale=1)
#             #q1 = ax.quiver(0, 0,  lims[0], 0, scale = .025)
#             #q2 = ax.quiver(0, 0, 0,lims[1], scale = .025)
#             ax.text(x=lims[0], y=0, s=r"$\phi_1$ ", fontsize=25)
#             ax.text(x=0, y=lims[1], s=r"$\phi_2$ ", fontsize=25)
#             if cbar == True:
#                 fig.colorbar(cax)
#             fig.savefig(filename, bbox_inches='tight', pad_inches=0, dpi=200)
#         if len(axes) == 3:

#             x = data[:, axes[0]][selected_points]
#             y = data[:, axes[1]][selected_points]
#             z = data[:, axes[2]][selected_points]
#             fig = plt.figure(figsize=(15, 10))
#             ax = fig.add_subplot(1, 1, 1, projection='3d')
#             coords = np.identity(3)
#             cax = ax.scatter(x, y, z, c=c, s=s, alpha=alpha, marker='.')
#             x2, y2, _ = proj3d.proj_transform(x, y, z, ax.get_proj())
#             lims = np.max(data, axis=0)
#             coords[0, 0] = lims[0]
#             coords[1, 1] = lims[1]
#             coords[2, 2] = lims[2]
#             # coord_proj_x, coord_proj_y, _ = proj3d.proj_transform(coords[:,0],coords[:,1],coords[:,2], ax.get_proj())
#             coord_proj_x, coord_proj_y, _ = proj3d.proj_transform(coords[:, 0], coords[:, 1], coords[:, 2],
#                                                                   ax.get_proj())

#             fig2 = plt.figure(figsize=plt.figaspect(.5))
#             ax2 = fig2.add_subplot(1, 1, 1)
#             cax2 = ax2.scatter(x2, y2, c=c, s=s, alpha=alpha, marker='.')
#             ax2.set_axis_off()
#             q1 = ax2.quiver(0, 0, coord_proj_x[0], coord_proj_y[0], scale=1, units='xy', angles='xy', scale_units='xy')
#             q2 = ax2.quiver(0, 0, coord_proj_x[1], coord_proj_y[1], scale=1, units='xy', angles='xy', scale_units='xy')
#             q3 = ax2.quiver(0, 0, coord_proj_x[2], coord_proj_y[2], scale=1, units='xy', angles='xy', scale_units='xy')
#             ax2.text(x=coord_proj_x[0], y=coord_proj_y[0], s=r"$\phi_1$ ", fontsize=25)
#             ax2.text(x=coord_proj_x[1], y=coord_proj_y[1], s=r"$\phi_2$ ", fontsize=25)
#             ax2.text(x=coord_proj_x[2], y=coord_proj_y[2], s=r"$\phi_3$ ", fontsize=25)
#             if cbar == True:
#                 fig2.colorbar(cax2)
#             fig2.savefig(filename, bbox_inches='tight', pad_inches=0, dpi=200)


    def get_embedding3(self, geom, n_components, diffusion_time , d):

        Y0, eigenvalues, eigenvectors  = spectral_embedding.spectral_embedding(geom=geom, eigen_solver='amg', random_state=6, diffusion_maps=True, diffusion_time = diffusion_time, n_components=n_components)
        #geom.rmetric = RiemannMetric(Y0,geom.laplacian_matrix,n_dim=n_components)
        geom.rmetric = RiemannMetric(Y0,geom.laplacian_matrix,n_dim=d)
        geom.rmetric.get_rmetric()
        output = RiemannianManifold(Y0, d)
        output.geom = geom
        return(output)

    def plot_induced_rmetric(self, axes, selected_points,s,alpha):
        data = self.data
        geom = self.geom

        svals = geom.rmetric.Gsvals
        n_svals = geom.rmetric.Gsvals.shape[1]
        if len(axes) == 2:
            2+2
        if len(axes) == 3:
            for j in range(n_svals):
                fig = plt.figure(figsize=(20,8))
                ax = fig.add_subplot(1, (n_svals + 1), (j+1), projection='3d')
                cax = ax.scatter(data[:, axes[0]][selected_points], data[:, axes[1]][selected_points], data[:,axes[2]][selected_points], c = svals[:,j][selected_points], s=  s, alpha=alpha, marker = '.')
                ax.set_axis_off()
                fig.colorbar(cax)

    def compute_nbr_wts(self, A, sample):
        Ps = list()
        nbrs = list()
        for ii in range(len(sample)):
            w = np.array(A[sample[ii],:].todense()).flatten()
            p = w / np.sum(w)
            nbrs.append(np.where(p > 0)[0])
            Ps.append(p[nbrs[ii]])
        return(Ps, nbrs)

    def get_wlpca_tangent_sel(self, M, selectedpoints, dim = None):

        n = self.n
        nsel = len(selectedpoints)
        if dim == None:
            dim = self.dim
        data = M.data
        A = M.geom.affinity_matrix
        (PS, nbrs) = self.compute_nbr_wts(A, selectedpoints)
        d = M.data.shape[1]
        tangent_bases = np.zeros((nsel, d, dim))
        for i in range(nsel):
            # print(i)
            p = PS[i]
            nbr = nbrs[i]
            Z = (data[nbr, :] - np.dot(p, data[nbr, :])) * p[:, np.newaxis]
            sig = np.dot(Z.transpose(), Z)
            e_vals, e_vecs = np.linalg.eigh(sig)
            j = e_vals.argsort()[::-1]  # Returns indices that will sort array from greatest to least.
            e_vec = e_vecs[:, j]
            e_vec = e_vec[:, :dim]
            tangent_bases[i, :, :] = e_vec
        return (tangent_bases)

    def __copy__(self):
        return(RiemannianManifold(self.data, self.dim))

In [ ]:
#export 
import numpy as np
import matplotlib.pyplot as plt
import os
from montlake.geometry.geometry import RiemannianManifold

class TangentBundle(RiemannianManifold):

    """
    This class stores data points, bases of their tangent space, and the riemannian metric induced by the positions of the points
    Parameters
    ----------
    embedding : int
        Number of samples
    tangent_bases : 

    Methods
    -------
    get_local_rmetric(self, embedding, tangent_bases)
        estimates rmetric in given embedding and tangent_bases
    """
    def __init__(self, manifold, tangent_bases):
        self.manifold = manifold
        self.tangent_bases = tangent_bases

    def plot_tangent_bases(self, show_pts, arrow_pts, axes, tangent_axes, arrow_scaler = .1):
        data = self.manifold.data
        tangent_bases = self.tangent_bases
        dim  = self.manifold.dim
        #colors = new color for every tan basis vector
        colors = ['r','g']
        if len(axes) == 3:
            fig = plt.figure(figsize=plt.figaspect(.5))
            ax = fig.add_subplot(1, 1,1, projection='3d')
            cax = ax.scatter(data[show_pts, 0], data[show_pts, 1], data[show_pts,2], s=  .5, alpha=.1, marker = '.', cmap = parula_map)
            for i in arrow_pts:
                for j in tangent_axes:
                    #print(i)
                    X,Y,Z = data[i,:]
                    U,V,W = tangent_bases[i,:,j] * arrow_scaler
                    ax.quiver(X,Y,Z,U,V,W, alpha = .5, color = colors[j])
                    #U, V, W = tangent_bases[i,:,1] * arrow_scaler
                    #ax.quiver(X,Y,Z,U,V,W, alpha = .5, color = 'r')
            ax.set_axis_off()
        if len(axes) == 2:
            2+2
        #fig.colorbar(cax)

    def plot_local_riemann_metric(self, axes, selectedpoints):
        data = self.manifold.data
        #G_sval = self.G_sval

        if len(axes == 3):
            fig = plt.figure(figsize=plt.figaspect(.5))
            for k in range(dim):
                ax = fig.add_subplot(dim, 1,(k+1) , projection='3d')
                cax = ax.scatter(data[selectedpoints, 0], data[selectedpoints, 1], data[selectedpoints,2], s=  .5, alpha=1, marker = '.', c = G_sval[selectedpoints,k], cmap = parula_map)
                ax.set_axis_off()
                fig.colorbar(cax)
        if len(axes == 2):
            2+2

    #these should be moved to the base class
    def get_local_riemann_metric(self, tangent_bases, embedding):
        n = self.manifold.data.shape[0]
        dim = self.manifold.dim
        affinity_matrix = self.geom.affinity_matrix
        
        metric = np.zeros((n, dim, dim))
        dualmetric = np.zeros((n, dim, dim))
        G_sval = np.zeros((n, dim))
        H_sval = np.zeros((n, dim))
        for i in range(n):
            nnebs = len(affinity_matrix[i].indices)
            nebs = affinity_matrix[0:nnebs]
            projected_positions = {}
            projected_positions[i] = np.matmul(tangent_bases[i,:,:].transpose(), (embedding[nebs,:] - embedding[i,:]).transpose())
            laplacian_pt = affinity_matrix[i].copy()
            pp = projected_positions[i].transpose()
            nbr = laplacian_pt.indices[1:]
            dualmetric[i,:,:], metric[i,:,:], G_sval[i,:], H_sval[i,:] = self._local_riemann_metric(pp, laplacian_pt)

        return(dualmetric, metric, G_sval, H_sval)

    def _local_riemann_metric(self, pp, laplacian_pt):
        dim = self.dim
        
        n_neighbors = len(laplacian_pt.indices)
        h_dual_metric = np.zeros((dim, dim ))
        lp = laplacian_pt.tocsr()
        nebs = laplacian_pt.indices[0:len(laplacian_pt.indices)]
        lpnebs = lp[:,nebs]
        for i in np.arange(dim):
            for j in np.arange(i,dim):
                #print(lpnebs.shape, pp.shape)
                h_dual_metric[i, j] = 0.5*lpnebs.multiply(pp[:,j]).multiply(pp[:,i]).sum()
        for j in np.arange(dim-1):
            for i in np.arange(j+1,self.dim):
                h_dual_metric[i,j] = h_dual_metric[j,i]

        # compute rmetric if requested
        riemann_metric = np.linalg.inv(h_dual_metric)
        return h_dual_metric, riemann_metric, np.linalg.svd(riemann_metric)[1], np.linalg.svd(h_dual_metric)[1]


    def __copy__(self):
        return(TangentBundle(self.manifold, self.tangent_bases))

 